In [2]:
from datasets import load_dataset

mbpp = load_dataset("mbpp")

In [138]:
mbpp['test'][0]

{'task_id': 11,
 'text': 'Write a python function to remove first and last occurrence of a given character from the string.',
 'code': 'def remove_Occ(s,ch): \r\n    for i in range(len(s)): \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    for i in range(len(s) - 1,-1,-1):  \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    return s ',
 'test_list': ['assert remove_Occ("hello","l") == "heo"',
  'assert remove_Occ("abcda","a") == "bcd"',
  'assert remove_Occ("PHP","P") == "H"'],
 'test_setup_code': '',
 'challenge_test_list': ['assert remove_Occ("hellolloll","l") == "helollol"',
  'assert remove_Occ("","l") == ""']}

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

def load_model_and_tokenizer(model_name = "google/gemma-2b-it", device = None):
    
    if device is None:
      device = 'cpu'
    
    tokenizer = AutoTokenizer.from_pretrained(model_name,
                                              trust_remote_code=True,
                                              )
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device, torch_dtype=torch.bfloat16)
    model.eval()
    
    return tokenizer, model

In [4]:
import warnings

import numpy as np

import torch

tokenizer, model = load_model_and_tokenizer('meta-llama/Llama-3.2-1B-Instruct')

In [116]:
import ast

def predict_code(model, tokenizer, problem_description):

    prompt = [{"role":"user", "content": f"Problem: {problem_description}\n\n"
            "Input:\nWrite a single Python function to solve the problem above.\nOutput:\n"}] 

    inputs = tokenizer.apply_chat_template(prompt, return_tensors="pt", add_generation_prompt=True)

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1
    )
    print(outputs)
    predicted_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return predicted_code[len(prompt):]

def parse_test_case(test_case):
    """
    Parse an assertion like:
    assert max_chain_length([Pair(1, 2), Pair(3, 4)], 4) == 4
    to extract:
    - Function call as a string: "max_chain_length([Pair(1, 2), Pair(3, 4)], 4)"
    - Expected output: 4
    """
    # Parse the assertion string
    assertion_node = ast.parse(test_case, mode='exec').body[0]
    if isinstance(assertion_node, ast.Assert):
        test_call = assertion_node.test.left  # The function call (left of ==)
        # i want to take out the name of the function too
        test_call = test_call.args
        expected_output = assertion_node.test.comparators[0]  # The value (right of ==)

        # Convert AST back to source code for evaluation
        test_call_code = ast.unparse(test_call)
        expected_output_value = eval(ast.unparse(expected_output))
        return test_call_code, expected_output_value
    else:
        raise ValueError("Test case is not a valid assertion.")
    

def evaluate_problem(function, test_cases):

    success_count = 0
    local_env = {}
    exec(function, {}, local_env)
    function_handle = next(iter(local_env.values()))
    for test_case in test_cases:
        input_data, expected_output = parse_test_case(test_case)
        try:
            result = function_handle(eval(input_data))
            if result == expected_output:
                success_count += 1
        except Exception as e:
            print("not a python code")
            continue

    # Return results
    accuracy = success_count / len(test_cases)
    print(accuracy)
    return accuracy




In [94]:
def clean_code(predicted_code):
    """
    Finds the first occurrence of 'def' in the predicted code and removes everything before it.
    If 'def' is not found, returns the code unchanged.
    """
    def_index = predicted_code.find("def")  # Find the index of the first 'def'
    if def_index != -1:
        return predicted_code[def_index:]  # Return the substring starting from 'def'
    return predicted_code  
for i in range(0, 10):
    print("Problem", i)
    sample_problem = mbpp['train'][i] 
    problem_description = sample_problem["text"]
    ground_truth_code = sample_problem["code"]
    test_cases = sample_problem["test_list"]

    predicted_code = predict_code(model, tokenizer, problem_description)
    print("pred:", predicted_code)
    print("ground:", ground_truth_code)
    cleaned_code = clean_code(predicted_code)
    print("cleaned:", cleaned_code)
    accuracy = evaluate_problem(cleaned_code, test_cases)
    print(accuracy)
# sample_problem = mbpp['train'][12] 
# problem_description = sample_problem["text"]
# ground_truth_code = sample_problem["code"]
# test_cases = sample_problem["test_list"]


# predicted_code = predict_code(model, tokenizer, problem_description)
# accuracy = evaluate_problem(predicted_code, test_cases)
# print(f"Problem Description:\n{problem_description}")
# print(f"Accuracy: {accuracy * 100:.2f}%")
# print(f"Predicted Code:\n{predicted_code}")
# print(f"Ground Truth Code:\n{ground_truth_code}")

Problem 0


TypeError: transformers.generation.utils.GenerationMixin.generate() argument after ** must be a mapping, not Tensor

In [ ]:
i = 12
print("Problem", i)
sample_problem = mbpp['train'][i] 
problem_description = sample_problem["text"]
ground_truth_code = sample_problem["code"]
test_cases = sample_problem["test_list"]

# predicted_code = predict_code(model, tokenizer, problem_description)
# print("pred:", predicted_code)
# print("ground:", ground_truth_code)

# extract code inside ```python
import re
def extract_function(predicted_code):
    pattern = r"```python\n(.*)\n```"
    match = re.findall(pattern, predicted_code, re.DOTALL)
    return match[-1]

print("cleaned:\n\n", extract_function(predicted_code))

#execute the code
#result = evaluate_problem(ground_truth_code, test_cases)
#print("v")
#result = evaluate_problem(extract_function(predicted_code), test_cases)

print(result)

# cleaned_code = extract_function(predicted_code)
# print("cleaned:", cleaned_code)
# accuracy = evaluate_problem(cleaned_code, test_cases)
# print(accuracy)

Problem 12
cleaned:

 def max_record_value(tuples_list):
    """
    This function finds the maximum value in a list of tuples.

    Args:
    tuples_list (list): A list of tuples.

    Returns:
    tuple: The maximum value in the list of tuples.
    """
    if not tuples_list:
        return None  # Return None if the list is empty

    max_value = max(tuples_list[0])  # Initialize max_value with the first tuple
    for tuple in tuples_list[1:]:  # Iterate over the rest of the tuples
        if tuple[0] > max_value:  # Compare the first element of the current tuple with max_value
            max_value = tuple[0]  # Update max_value if the current tuple's first element is larger

    return max_value  # Return the maximum value


# Example usage
tuples_list = [(10, 20, 30), (40, 50, 60), (70, 80, 90)]
max_value = max_record_value(tuples_list)
print("Maximum value:", max_value)  # Output: Maximum value: (90, 60, 70)
1.0
v
Maximum value: 70
not a python code
not a python code
not a pytho

In [132]:
print(pred)

def max_record_value(tuples_list):
    """
    This function finds the maximum value in a list of tuples.

    Args:
    tuples_list (list): A list of tuples.

    Returns:
    tuple: The maximum value in the list of tuples.
    """
    if not tuples_list:
        return None  # Return None if the list is empty

    max_value = max(tuples_list[0])  # Initialize max_value with the first tuple
    for tuple in tuples_list[1:]:  # Iterate over the rest of the tuples
        if tuple[0] > max_value:  # Compare the first element of the current tuple with max_value
            max_value = tuple[0]  # Update max_value if the current tuple's first element is larger

    return max_value  # Return the maximum value


# Example usage
tuples_list = [(10, 20, 30), (40, 50, 60), (70, 80, 90)]
max_value = max_record_value(tuples_list)
print("Maximum value:", max_value)  # Output: Maximum value: (90, 60, 70)


In [1]:
from vllm import SamplingParams
import re

def extract_python(predicted_code):
    pattern = r"```python\n(.*)\n```"
    match = re.findall(pattern, predicted_code, re.DOTALL)
    return match[-1]

def run_mbpp_benchmark(model, prompt_type, max_tokens=1024):
    import datasets
    data = datasets.load_dataset("mbpp")['test']
    
    inputs = []
    test_cases = []
    for i in range(len(data)):
        inputs.append([{ "role": "user", "content": data[i]['text']}])
        test_cases.append("\n".join(data[i]['challenge_test_list']))

    if prompt_type == 'chat':
        inputs = model.get_tokenizer().apply_chat_template(inputs, add_generation_prompt=True, tokenize=False)
    elif prompt_type == 'plain':
        pass
    else:
        raise NotImplementedError('problem_prompt should be either "chat" or "plain"')
    

    sampling = SamplingParams(n=1, temperature=0.0, max_tokens=max_tokens, top_p=1.0)
    
    results = model.generate(inputs, sampling_params=sampling)
    results = extract_outputs(results)

    accuracy = 0
    for i in range(len(results)):
        predicted_code = results[i]
        cleaned_code = extract_python(predicted_code)
        try:
            exec(cleaned_code+'\n'+test_cases[i])
            accuracy += 1
        except:
            pass
    
    return accuracy / len(results)

def extract_outputs(outputs):
    return [output.outputs[0].text for output in outputs]

In [3]:
import torch

In [2]:
from EvalUtils import load_model

model = load_model('meta-llama/Llama-3.2-1B-Instruct')

INFO 12-06 22:24:24 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-06 22:24:24 config.py:1020] Defaulting to use mp for distributed inference
WARNING 12-06 22:24:24 arg_utils.py:1013] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-06 22:24:24 config.py:1136] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-06 22:24:24 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=39808, download_dir=Non

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(VllmWorkerProcess pid=171550) INFO 12-06 22:24:26 weight_utils.py:288] No model.safetensors.index.json found in remote.
INFO 12-06 22:24:26 model_runner.py:1077] Loading model weights took 1.1666 GB
(VllmWorkerProcess pid=171550) INFO 12-06 22:24:26 model_runner.py:1077] Loading model weights took 1.1666 GB
(VllmWorkerProcess pid=171550) INFO 12-06 22:24:26 worker.py:232] Memory profiling results: total_gpu_memory=23.53GiB initial_memory_usage=7.40GiB peak_torch_memory=1.21GiB memory_usage_post_profile=7.60GiB non_torch_memory=6.42GiB kv_cache_size=13.55GiB gpu_memory_utilization=0.90
INFO 12-06 22:24:27 worker.py:232] Memory profiling results: total_gpu_memory=23.54GiB initial_memory_usage=16.61GiB peak_torch_memory=2.34GiB memory_usage_post_profile=16.81GiB non_torch_memory=15.64GiB kv_cache_size=3.21GiB gpu_memory_utilization=0.90
INFO 12-06 22:24:27 distributed_gpu_executor.py:57] # GPU blocks: 13153, # CPU blocks: 16384
INFO 12-06 22:24:27 distributed_gpu_executor.py:61] Maximum 

AttributeError: 'PyNcclCommunicator' object has no attribute 'device'

In [ ]:
run_mbpp_benchmark(model, 'chat', max_tokens=1024)

INFO 12-06 22:22:52 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-06 22:22:52 config.py:1020] Defaulting to use mp for distributed inference
WARNING 12-06 22:22:52 arg_utils.py:1013] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-06 22:22:52 config.py:1136] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-06 22:22:52 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=39808, download_dir=Non

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(VllmWorkerProcess pid=170833) INFO 12-06 22:22:53 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 12-06 22:22:54 utils.py:961] Found nccl from library libnccl.so.2
ERROR 12-06 22:22:54 pynccl_wrapper.py:196] Failed to load NCCL library from libnccl.so.2 .It is expected if you are not running on NVIDIA/AMD GPUs.Otherwise, the nccl library might not exist, be corrupted or it does not support the current platform Linux-6.5.0-35-generic-x86_64-with-glibc2.35.If you already have the library, please set the environment variable VLLM_NCCL_SO_PATH to point to the correct nccl library path.
(VllmWorkerProcess pid=170833) INFO 12-06 22:22:54 utils.py:961] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=170833) ERROR 12-06 22:22:54 pynccl_wrapper.py:196] Failed to load NCCL library from libnccl.so.2 .It is expected if you are not running on NVIDIA/AMD GPUs.Otherwise, the nccl library might not exist, be corrupted or it does not support the current platform Linux-6.5.

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(VllmWorkerProcess pid=170833) INFO 12-06 22:23:03 weight_utils.py:288] No model.safetensors.index.json found in remote.
INFO 12-06 22:23:03 model_runner.py:1077] Loading model weights took 1.1666 GB
(VllmWorkerProcess pid=170833) INFO 12-06 22:23:03 model_runner.py:1077] Loading model weights took 1.1666 GB
(VllmWorkerProcess pid=170833) INFO 12-06 22:23:04 worker.py:232] Memory profiling results: total_gpu_memory=23.53GiB initial_memory_usage=7.40GiB peak_torch_memory=1.21GiB memory_usage_post_profile=7.60GiB non_torch_memory=6.42GiB kv_cache_size=13.55GiB gpu_memory_utilization=0.90
INFO 12-06 22:23:04 worker.py:232] Memory profiling results: total_gpu_memory=23.54GiB initial_memory_usage=16.61GiB peak_torch_memory=2.34GiB memory_usage_post_profile=16.81GiB non_torch_memory=15.64GiB kv_cache_size=3.21GiB gpu_memory_utilization=0.90
INFO 12-06 22:23:04 distributed_gpu_executor.py:57] # GPU blocks: 13153, # CPU blocks: 16384
INFO 12-06 22:23:04 distributed_gpu_executor.py:61] Maximum 

AttributeError: 'PyNcclCommunicator' object has no attribute 'device'

(VllmWorkerProcess pid=170833) ERROR 12-06 22:23:06 multiproc_worker_utils.py:229]     self.model_runner.capture_model(self.gpu_cache)
(VllmWorkerProcess pid=170833) ERROR 12-06 22:23:06 multiproc_worker_utils.py:229]   File "/home/amin/miniforge3/envs/NLP/lib/python3.12/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
(VllmWorkerProcess pid=170833) ERROR 12-06 22:23:06 multiproc_worker_utils.py:229]     return func(*args, **kwargs)
(VllmWorkerProcess pid=170833) ERROR 12-06 22:23:06 multiproc_worker_utils.py:229]            ^^^^^^^^^^^^^^^^^^^^^
(VllmWorkerProcess pid=170833) ERROR 12-06 22:23:06 multiproc_worker_utils.py:229]   File "/home/amin/miniforge3/envs/NLP/lib/python3.12/site-packages/vllm/worker/model_runner.py", line 1508, in capture_model
(VllmWorkerProcess pid=170833) ERROR 12-06 22:23:06 multiproc_worker_utils.py:229]     graph_runner.capture(**capture_inputs)
(VllmWorkerProcess pid=170833) ERROR 12-06 22:23:06 multiproc_worker_utils.py:229]   Fil

In [133]:
full_code = pred + "\n" + "\n".join(test_cases)

# run the code
try:
    exec(full_code)
except Exception as e:
    print(e)
    print("error")

Maximum value: 70


In [119]:
pred = extract_function(predicted_code)

local_env = {}
exec(pred, {}, local_env)
function_handle = next(iter(local_env.values()))
test_case = test_cases[0]
input_data, expected_output = parse_test_case(test_case)
print(function_handle)
result = function_handle(eval(input_data))
print(result)

Maximum value: 70
<function max_record_value at 0x797d148c85e0>


TypeError: '>' not supported between instances of 'list' and 'str'

In [52]:
prompt = (f"Problem: {problem_description}\n\n""Input:\nWrite a Python function to solve the problem above.\nOutput:\n")   

In [29]:
def extract_python_code_block(predicted_code):
    """
    Extracts the Python code block enclosed between ```python and ```.
    """
    start_tag = "```python"
    end_tag = "```"
    
    # Find the first Python code block
    start_index = predicted_code.find(start_tag)
    end_index = predicted_code.find(end_tag, start_index + len(start_tag))

    if start_index != -1 and end_index != -1:
        # Extract and return the content inside the code block
        return predicted_code[start_index + len(start_tag):end_index].strip()
    
    return "No Python code block found."

In [92]:
def evaluate_problem(function, test_cases):

    success_count = 0
    local_env = {}
    exec(function, {}, local_env)
    function_handle = next(iter(local_env.values()))
    for test_case in test_cases:
        input_data, expected_output = parse_test_case(test_case)
        try:
            result = function_handle(eval(input_data))
            print("result:", result)
            if result == expected_output:
                success_count += 1
        except Exception as e:
            print("not a python code")
            continue

    # Return results
    accuracy = success_count / len(test_cases)
    return accuracy


evaluate_problem(ground_truth_code, test_cases)

result: [('key1', 5), ('key2', 4), ('key3', 9)]
result: [('key1', 6), ('key2', 5), ('key3', 10)]
result: [('key1', 7), ('key2', 6), ('key3', 11)]


1.0

In [ ]:


def parse_test_case(test_case):
    """
    Parse an assertion like:
    assert max_chain_length([Pair(1, 2), Pair(3, 4)], 4) == 4
    to extract:
    - Function call as a string: "max_chain_length([Pair(1, 2), Pair(3, 4)], 4)"
    - Expected output: 4
    """
    # Parse the assertion string
    assertion_node = ast.parse(test_case, mode='exec').body[0]
    if isinstance(assertion_node, ast.Assert):
        test_call = assertion_node.test.left  # The function call (left of ==)
        # i want to take out the name of the function too
        test_call = test_call.args
        expected_output = assertion_node.test.comparators[0]  # The value (right of ==)

        # Convert AST back to source code for evaluation
        test_call_code = ast.unparse(test_call)
        expected_output_value = eval(ast.unparse(expected_output))
        return test_call_code, expected_output_value
    else:
        raise ValueError("Test case is not a valid assertion.")

("[('key1', [3, 4, 5]), ('key2', [1, 4, 2]), ('key3', [9, 3])]", [('key1', 5), ('key2', 4), ('key3', 9)])
("[('key1', [4, 5, 6]), ('key2', [2, 5, 3]), ('key3', [10, 4])]", [('key1', 6), ('key2', 5), ('key3', 10)])
("[('key1', [5, 6, 7]), ('key2', [3, 6, 4]), ('key3', [11, 5])]", [('key1', 7), ('key2', 6), ('key3', 11)])


In [36]:
cleaned_code = extract_python_code_block(predicted_code)
print(cleaned_code)
accuracy = evaluate_problem(cleaned_code, test_cases)
print(accuracy)

def find_max_value_in_list(tuple_list):
    """
    This function finds the maximum value in a list of tuples.

    Parameters:
    tuple_list (list): A list of tuples.

    Returns:
    tuple: The maximum value in the list of tuples.
    """
    if not tuple_list:
        return None
    max_value = max(tuple_list, key=lambda x: x[1])
    return max_value

# Example usage:
tuple_list = [(1, 2), (3, 4), (5, 6), (7, 8)]
max_value = find_max_value_in_list(tuple_list)
print(max_value)  # Output: (7, 8)
(7, 8)
not a python code
(7, 8)
not a python code
(7, 8)
not a python code
0.0


In [ ]:
c = """def find_max_value_in_list(tuple_list):
    \"\"\"
    This function finds the maximum value in a list of tuples.

    Parameters:
    tuple_list (list): A list of tuples.

    Returns:
    tuple: The maximum value in the list of tuples.
    \"\"\"
    if not tuple_list:
        return None
    max_value = max(tuple_list, key=lambda x: x[1])
    return max_value
    
    """

In [96]:
i = 12
print("Problem", i)
sample_problem = mbpp['train'][i] 
problem_description = sample_problem["text"]
ground_truth_code = sample_problem["code"]
test_cases = sample_problem["test_list"]
input_data, expected_output = parse_test_case(test_cases[0])
print(input_data, expected_output)

Problem 12
[('key1', [3, 4, 5]), ('key2', [1, 4, 2]), ('key3', [9, 3])] [('key1', 5), ('key2', 4), ('key3', 9)]


In [49]:
print(ground_truth_code)

def maximum_value(test_list):
  res = [(key, max(lst)) for key, lst in test_list]
  return (res) 


In [58]:



for test_case in test_cases:
    input_data = '''[('key1', [3, 4, 5]), ('key2', [1, 4, 2]), ('key3', [9, 3])]'''
    expected_output = "[('key1', 5), ('key2', 4), ('key3', 9)]"
    #input_data, expected_output = parse_test_case(test_case)
    # Use `exec` to run the code and check the output
    local_env = {}

    exec(ground_truth_code, {}, local_env)
    result = local_env[input_data]
    print(result)

    exec(c, {}, local_env)
    result = local_env[input_data]
    print(result)


KeyError: "[('key1', [3, 4, 5]), ('key2', [1, 4, 2]), ('key3', [9, 3])]"

In [61]:
print(ground_truth_code)

def maximum_value(test_list):
  res = [(key, max(lst)) for key, lst in test_list]
  return (res) 


In [ ]:
local_env = {}

try:
    # Dynamically execute the function string
    exec(ground_truth_code, {}, local_env)

    # Extract the function name (assumes it's the first 'def' in the string)
    function_name = ground_truth_code.split("def ")[1].split("(")[0].strip()

    # Call the function dynamically with the input
    result = local_env[function_name](input_data)
    print(result)
except Exception as e:
    print(f"Error during function execution: {e}")

Error during function execution: not enough values to unpack (expected 2, got 1)


In [83]:
exec(ground_truth_code, {}, local_env)

In [ ]:
exec(ground_truth_code, {}, local_env)
function_handle = next(iter(local_env.values()))
result = function_handle(eval(input_data))
print(result)

In [78]:
result = function_handle(eval(input_data))
print(result)

[('key1', 5), ('key2', 4), ('key3', 9)]


In [24]:
def extract_function(predicted_code):
    """
    Extracts a function from a code string starting with 'def' and ending based on indentation rules.
    """
    lines = predicted_code.splitlines()  # Split code into lines
    start_index = -1

    # Find the first occurrence of a line starting with 'def'
    for i, line in enumerate(lines):
        if line.strip().startswith("def "):  # Check for the 'def' keyword
            start_index = i
            break

    if start_index == -1:
        return "No function definition found."

    # Determine indentation of the function header
    function_indent = len(lines[start_index]) - len(lines[start_index].lstrip())
    function_lines = [lines[start_index]]  # Start collecting the function from 'def'

    # Collect all indented lines after the function header
    for line in lines[start_index + 1:]:
        current_indent = len(line) - len(line.lstrip())

        # Stop if indentation decreases or line is blank and isn't a continuation
        if current_indent <= function_indent:
            break

        function_lines.append(line)

    # Join the collected lines to reconstruct the function
    return "\n".join(function_lines)

In [57]:
def maximum_value(test_list):
  res = [(key, max(lst)) for key, lst in test_list]
  return (res) 

maximum_value([("key1", [3, 4, 5]), ("key2", [1, 4, 2]), ("key3", [9, 3])])

[('key1', 5), ('key2', 4), ('key3', 9)]

In [97]:
print(predicted_code)

Write a single Python function to solve the problem above.

```python
def find_max_value_in_list(tuple_list):
    """
    This function finds the maximum value in a list of tuples.

    Parameters:
    tuple_list (list): A list of tuples.

    Returns:
    tuple: The maximum value in the list of tuples.
    """
    if not tuple_list:
        return None
    max_value = max(tuple_list, key=lambda x: x[1])
    return max_value

# Example usage:
tuple_list = [(1, 2), (3, 4), (5, 6), (7, 8)]
max_value = find_max_value_in_list(tuple_list)
print(max_value)  # Output: (7, 8)
```

This solution works by using the built-in `max` function in Python, which returns the largest item in an iterable. The `key` argument of the `max` function is used to specify that we want to compare the tuples based on their values (i.e., the second element of each tuple). The `max_value` variable is assigned the maximum tuple found in the list. Finally, the function returns `max_value`. The example usage demonstrate